# Import GeoPackage into Delta Lake with DuckDB

_NOTE: This example is focusing on using DuckDB to parse the GeoPackage. For more complex GeoPackages, you may need to install GDAL yourself and use the GDAL command line tools._

This example in facts works just as much for any spatial formats that DuckDB Spatial supports via its GDAL integration, see the output of [ST_Drivers](https://duckdb.org/docs/stable/core_extensions/spatial/functions.html#st_drivers).

In [ ]:
import duckdb

duckdb.sql("install spatial; load spatial")

In [ ]:
GPKG_URL = "https://service.pdok.nl/kadaster/bestuurlijkegebieden/atom/v1_0/downloads/BestuurlijkeGebieden_2025.gpkg"

In [ ]:
layers = duckdb.sql(
    f"""
with t as (
    select unnest(layers) layer
     from st_read_meta('{GPKG_URL}'))
select
    layer.name layer_name,
    layer.geometry_fields[1].name geom_field
from t"""
).df()

layers

# Returns:

# layer_name	geom_field
# 0	gemeentegebied	geom
# 1	landgebied	geom
# 2	provinciegebied	geom

In [ ]:
layer_name, geom_field = layers.loc[0, ["layer_name", "geom_field"]]

duckdb.sql(
    f"""copy (
  select * replace(st_aswkb({geom_field}) as {geom_field})
  from
    st_read(
      '{GPKG_URL}',
      layer='{layer_name}')
  ) to '/tmp/{layer_name}.parquet' (format parquet)"""
)

In [ ]:
spark.read.parquet("/tmp/{layer_name}.parquet").display()  # noqa: S108

You can store the above spark data frame as a Delta Lake table as needed.